In [1]:
import pandas as pd
import numpy as np
from dotenv import load_dotenv
import os
from openai import OpenAI
import streamlit as st
import spotipy 

# Cargar variables de entorno desde un archivo .env
load_dotenv()

# Importar funciones de gestión
from utils.chat_gpt_manager import (
    get_chords_chat,
    create_song
)
from utils.spotify_manager import (
    get_similar_songs,
    get_audio_analysis,
    get_audio_features,
    get_song_and_artist
)
from utils.functions import (
    add_info_list_df,
    convert_chords_with_minor,
    standardize_chord_symbols,
    transpose_chords_to_key,
    check_track_csv,
    filter_existing_vector_id,
    to_embbed_text_column,
    expand_and_remove_original_columns,
    drop_columns_with_many_nulls,
    get_user_input,
    handle_input_search,
    calc_diff,
    export_unique_rows,
    create_metadata_column,
    clean_metadata_column,
    insert_into_pinecone,
    create_dataframe_for_pinecone,
    realizar_consulta

    
)

from utils.pinecone_manager import (
    initialize_pinecone, 
    setup_index
)

from utils.lyrics_manager import insert_lyrics_db

from pinecone import Pinecone, ServerlessSpec
from pinecone.grpc import PineconeGRPC as Pinecone

# Initizalize pinecone

In [2]:
pc = initialize_pinecone()

In [3]:
index = setup_index(pc)

Index 'song-starter-index' already exists, skipping creation.


In [4]:
index_name= pc.list_indexes().indexes[0].name

# API de Spotify

Aquí se utiliza la API de Spotify para obtener recomendaciones de canciones basadas en una canción dada. 

Input:

In [5]:
input_song, input_artist = get_user_input()

In [6]:
# Llamar a la función para obtener el track ID
input_track_id=handle_input_search(input_song, input_artist)

In [7]:
input_song , input_artist = get_song_and_artist(input_track_id)

In [8]:
input_data = pd.DataFrame({'id': [input_track_id]})
input_data['name'] = input_song
input_data['artists'] = input_artist

In [9]:
input_data

,id,name,artists
0,42VgiRyg0YjtRVLiPFNxPJ,Zapatillas,El Canto Del Loco


Similar songs

In [10]:
tracks_data = get_similar_songs(input_track_id, 5)

In [11]:
for track in tracks_data:
    print(track['name'])


Sera
Tanto la Queria
Kamikaze
Quiero tener tu presencia
Copacabana


In [12]:
tracks_data = pd.DataFrame(tracks_data)

In [13]:
tracks_data.head(2)

,id,name,artists
0,03CK09Va5nmnr4UwubyZtz,Sera,El Canto Del Loco
1,2VSGzQqy9nqJefcYGJNWst,Tanto la Queria,Andy & Lucas


MIRAR ESTO, cómo evitar que a partir de aquí todo de error si ya están todas en pinecone

In [14]:
filter_existing_vector_id(index, tracks_data)            

In [15]:

tracks_data.reset_index(drop=True, inplace=True)

In [16]:
tracks_data

,id,name,artists
0,2VSGzQqy9nqJefcYGJNWst,Tanto la Queria,Andy & Lucas
1,05iTWBqTlBZAYBzHql2B3Y,Kamikaze,Amaral
2,3f2k0XTc2ZXG17jEMDu8pp,Quiero tener tu presencia,Seguridad Social
3,4hTXeWayUVMwoz6v1CviSN,Copacabana,IZAL


In [17]:
csv= pd.read_csv('mi_archivo_temporal.csv')

In [18]:
if len(tracks_data)>0:
    df=check_track_csv(tracks_data, csv)

In [19]:
if len(df)>0:    
    df.reset_index(drop=True, inplace=True)

# API de ChatGPT

Aquí se utiliza la API de Chat GPT, para obtener los acordes e instrumentos de las canciones

Input:

In [20]:
add_info_list_df(input_data, get_chords_chat, 'chords', max_retries=3)

,id,name,artists,chords
0,42VgiRyg0YjtRVLiPFNxPJ,Zapatillas,El Canto Del Loco,"[C, G, Am, F, D, A, Bm, G]"


In [21]:
add_info_list_df(input_data, get_chords_chat, 'instruments', max_retries=3)

,id,name,artists,chords,instruments
0,42VgiRyg0YjtRVLiPFNxPJ,Zapatillas,El Canto Del Loco,"[C, G, Am, F, D, A, Bm, G]","[C, G, Am, F, F, G, C, Am]"


Related tracks:

In [22]:
add_info_list_df(df, get_chords_chat, 'chords', max_retries=3)

,id,name,artists,chords
0,2VSGzQqy9nqJefcYGJNWst,Tanto la Queria,Andy & Lucas,"[A, E, F#m, D, A, E, D, D]"
1,05iTWBqTlBZAYBzHql2B3Y,Kamikaze,Amaral,"[Am, G, F, C, C, G, Am, F]"
2,3f2k0XTc2ZXG17jEMDu8pp,Quiero tener tu presencia,Seguridad Social,"[C, G, Am, F, Dm, G, C, F]"
3,4hTXeWayUVMwoz6v1CviSN,Copacabana,IZAL,"[F, A, D, B, G, C, Em, D]"


In [23]:
add_info_list_df(df, get_chords_chat, 'instruments', max_retries=3)

,id,name,artists,chords,instruments
0,2VSGzQqy9nqJefcYGJNWst,Tanto la Queria,Andy & Lucas,"[A, E, F#m, D, A, E, D, D]","[C, G, Am, F, C, G, F, C]"
1,05iTWBqTlBZAYBzHql2B3Y,Kamikaze,Amaral,"[Am, G, F, C, C, G, Am, F]","[G, D, Em, C, C, G, Am, D]"
2,3f2k0XTc2ZXG17jEMDu8pp,Quiero tener tu presencia,Seguridad Social,"[C, G, Am, F, Dm, G, C, F]","[D, A, Bm, G, D, A, G, A]"
3,4hTXeWayUVMwoz6v1CviSN,Copacabana,IZAL,"[F, A, D, B, G, C, Em, D]","[C, G, Am, F, F, G, C, Am]"


# API de Spotify para nuevas columnas

Input:

In [24]:
get_audio_features(input_data)

,id,name,artists,chords,instruments,speechiness,instrumentalness
0,42VgiRyg0YjtRVLiPFNxPJ,Zapatillas,El Canto Del Loco,"[C, G, Am, F, D, A, Bm, G]","[C, G, Am, F, F, G, C, Am]",0.178,0


In [25]:
get_audio_analysis(input_data)

,id,name,artists,chords,instruments,speechiness,instrumentalness,key,mode,tempo,loudness
0,42VgiRyg0YjtRVLiPFNxPJ,Zapatillas,El Canto Del Loco,"[C, G, Am, F, D, A, Bm, G]","[C, G, Am, F, F, G, C, Am]",0.178,0,8,0,198.076,-2.382


In [26]:
insert_lyrics_db(input_data)

(                       id        name            artists  \
 0  42VgiRyg0YjtRVLiPFNxPJ  Zapatillas  El Canto Del Loco   
 
                        chords                 instruments  speechiness  \
 0  [C, G, Am, F, D, A, Bm, G]  [C, G, Am, F, F, G, C, Am]        0.178   
 
    instrumentalness  key  mode    tempo  loudness  \
 0                 0    8     0  198.076    -2.382   
 
                                                theme  
 0  Estoy cansado de salir de noche y ver siempre ...  ,
 'Estoy cansado de salir de noche y ver siempre la misma gente.\nEstoy flipando de que la gente se invente, cuente y luego reinvente.\nApotronado en el sofá de mi casa, vente se está caliente.\nAmaestrados vamos al mismo sitio todos aunque luego ni entres.\nAlucinando de que me miren de arriba a bajo como un delincuente.\nIntoxicado de que me pongan esa puta música indiferente.\n\n(ESTRIBILLO)')

In [27]:
input_data['chords'].dropna(inplace=True)
input_data = input_data[input_data['chords'].apply(lambda x: x != [])]

In [28]:
input_data

,id,name,artists,chords,instruments,speechiness,instrumentalness,key,mode,tempo,loudness,theme
0,42VgiRyg0YjtRVLiPFNxPJ,Zapatillas,El Canto Del Loco,"[C, G, Am, F, D, A, Bm, G]","[C, G, Am, F, F, G, C, Am]",0.178,0,8,0,198.076,-2.382,Estoy cansado de salir de noche y ver siempre ...


Related tracks:

In [29]:
get_audio_features(df)

,id,name,artists,chords,instruments,speechiness,instrumentalness
0,2VSGzQqy9nqJefcYGJNWst,Tanto la Queria,Andy & Lucas,"[A, E, F#m, D, A, E, D, D]","[C, G, Am, F, C, G, F, C]",0.0605,0.000000
1,05iTWBqTlBZAYBzHql2B3Y,Kamikaze,Amaral,"[Am, G, F, C, C, G, Am, F]","[G, D, Em, C, C, G, Am, D]",0.0398,0.000047
2,3f2k0XTc2ZXG17jEMDu8pp,Quiero tener tu presencia,Seguridad Social,"[C, G, Am, F, Dm, G, C, F]","[D, A, Bm, G, D, A, G, A]",0.0797,0.000000
3,4hTXeWayUVMwoz6v1CviSN,Copacabana,IZAL,"[F, A, D, B, G, C, Em, D]","[C, G, Am, F, F, G, C, Am]",0.0485,0.000005


In [30]:
get_audio_analysis(df)

,id,name,artists,chords,instruments,speechiness,instrumentalness,key,mode,tempo,loudness
0,2VSGzQqy9nqJefcYGJNWst,Tanto la Queria,Andy & Lucas,"[A, E, F#m, D, A, E, D, D]","[C, G, Am, F, C, G, F, C]",0.0605,0.000000,10,1,144.001,-5.718
1,05iTWBqTlBZAYBzHql2B3Y,Kamikaze,Amaral,"[Am, G, F, C, C, G, Am, F]","[G, D, Em, C, C, G, Am, D]",0.0398,0.000047,8,0,137.770,-6.605
2,3f2k0XTc2ZXG17jEMDu8pp,Quiero tener tu presencia,Seguridad Social,"[C, G, Am, F, Dm, G, C, F]","[D, A, Bm, G, D, A, G, A]",0.0797,0.000000,4,1,146.075,-5.597
3,4hTXeWayUVMwoz6v1CviSN,Copacabana,IZAL,"[F, A, D, B, G, C, Em, D]","[C, G, Am, F, F, G, C, Am]",0.0485,0.000005,9,0,144.924,-7.194


In [31]:
insert_lyrics_db(df)

Timeout o fallo! Usando GPT para obtener el tema para Andy & Lucas - Tanto la Queria


(                       id                       name           artists  \
 0  2VSGzQqy9nqJefcYGJNWst            Tanto la Queria      Andy & Lucas   
 1  05iTWBqTlBZAYBzHql2B3Y                   Kamikaze            Amaral   
 2  3f2k0XTc2ZXG17jEMDu8pp  Quiero tener tu presencia  Seguridad Social   
 3  4hTXeWayUVMwoz6v1CviSN                 Copacabana              IZAL   
 
                        chords                 instruments  speechiness  \
 0  [A, E, F#m, D, A, E, D, D]   [C, G, Am, F, C, G, F, C]       0.0605   
 1  [Am, G, F, C, C, G, Am, F]  [G, D, Em, C, C, G, Am, D]       0.0398   
 2  [C, G, Am, F, Dm, G, C, F]   [D, A, Bm, G, D, A, G, A]       0.0797   
 3   [F, A, D, B, G, C, Em, D]  [C, G, Am, F, F, G, C, Am]       0.0485   
 
    instrumentalness  key  mode    tempo  loudness  \
 0          0.000000   10     1  144.001    -5.718   
 1          0.000047    8     0  137.770    -6.605   
 2          0.000000    4     1  146.075    -5.597   
 3          0.000005    9     

In [32]:
df_vectorial= df.copy()
df_vectorial['chords'].dropna(inplace=True)
df_vectorial = df_vectorial[df_vectorial['chords'].apply(lambda x: x != [])]

# Separate columns

Input:

In [33]:
input_data

,id,name,artists,chords,instruments,speechiness,instrumentalness,key,mode,tempo,loudness,theme
0,42VgiRyg0YjtRVLiPFNxPJ,Zapatillas,El Canto Del Loco,"[C, G, Am, F, D, A, Bm, G]","[C, G, Am, F, F, G, C, Am]",0.178,0,8,0,198.076,-2.382,Estoy cansado de salir de noche y ver siempre ...


In [34]:
input_data['chords_1'] = input_data['chords'].apply(lambda x: x[:4]) 
input_data['chords_2'] = input_data['chords'].apply(lambda x: x[4:])

In [35]:
input_data

,id,name,artists,chords,instruments,speechiness,instrumentalness,key,mode,tempo,loudness,theme,chords_1,chords_2
0,42VgiRyg0YjtRVLiPFNxPJ,Zapatillas,El Canto Del Loco,"[C, G, Am, F, D, A, Bm, G]","[C, G, Am, F, F, G, C, Am]",0.178,0,8,0,198.076,-2.382,Estoy cansado de salir de noche y ver siempre ...,"[C, G, Am, F]","[D, A, Bm, G]"


In [36]:
input_data['chords_1'] = input_data['chords_1'].apply(standardize_chord_symbols)
input_data['chords_2'] = input_data['chords_2'].apply(standardize_chord_symbols)

In [37]:
input_data

,id,name,artists,chords,instruments,speechiness,instrumentalness,key,mode,tempo,loudness,theme,chords_1,chords_2
0,42VgiRyg0YjtRVLiPFNxPJ,Zapatillas,El Canto Del Loco,"[C, G, Am, F, D, A, Bm, G]","[C, G, Am, F, F, G, C, Am]",0.178,0,8,0,198.076,-2.382,Estoy cansado de salir de noche y ver siempre ...,"[C, G, Am, F]","[D, A, Bm, G]"


In [38]:
input_data['chords_1_numeric'] = input_data['chords_1'].apply(convert_chords_with_minor)
input_data['chords_2_numeric'] = input_data['chords_2'].apply(convert_chords_with_minor)

In [39]:
input_data

,id,name,artists,chords,instruments,speechiness,instrumentalness,key,mode,tempo,loudness,theme,chords_1,chords_2,chords_1_numeric,chords_2_numeric
0,42VgiRyg0YjtRVLiPFNxPJ,Zapatillas,El Canto Del Loco,"[C, G, Am, F, D, A, Bm, G]","[C, G, Am, F, F, G, C, Am]",0.178,0,8,0,198.076,-2.382,Estoy cansado de salir de noche y ver siempre ...,"[C, G, Am, F]","[D, A, Bm, G]","[0, 7, 21, 5]","[2, 9, 23, 7]"


In [40]:
col_chords = ['chords_1_numeric', 'chords_2_numeric']

# Aplicar la función a cada fila
for col in col_chords:
    input_data[f'normal_{col}'] = input_data.apply(lambda row: transpose_chords_to_key(row, col), axis=1)

input_data

,id,name,artists,chords,instruments,speechiness,instrumentalness,key,mode,tempo,loudness,theme,chords_1,chords_2,chords_1_numeric,chords_2_numeric,normal_chords_1_numeric,normal_chords_2_numeric
0,42VgiRyg0YjtRVLiPFNxPJ,Zapatillas,El Canto Del Loco,"[C, G, Am, F, D, A, Bm, G]","[C, G, Am, F, F, G, C, Am]",0.178,0,8,0,198.076,-2.382,Estoy cansado de salir de noche y ver siempre ...,"[C, G, Am, F]","[D, A, Bm, G]","[0, 7, 21, 5]","[2, 9, 23, 7]","[-8, -1, 13, -3]","[-6, 1, 15, -1]"


In [41]:
for col in col_chords:
    input_data[f'diff_{col}'] = input_data.apply(lambda row: calc_diff(row[col]), axis=1)

input_data


,id,name,artists,chords,instruments,speechiness,instrumentalness,key,mode,tempo,loudness,theme,chords_1,chords_2,chords_1_numeric,chords_2_numeric,normal_chords_1_numeric,normal_chords_2_numeric,diff_chords_1_numeric,diff_chords_2_numeric
0,42VgiRyg0YjtRVLiPFNxPJ,Zapatillas,El Canto Del Loco,"[C, G, Am, F, D, A, Bm, G]","[C, G, Am, F, F, G, C, Am]",0.178,0,8,0,198.076,-2.382,Estoy cansado de salir de noche y ver siempre ...,"[C, G, Am, F]","[D, A, Bm, G]","[0, 7, 21, 5]","[2, 9, 23, 7]","[-8, -1, 13, -3]","[-6, 1, 15, -1]","[7, 14, -16]","[7, 14, -16]"


In [42]:
input_data.replace('\n', ' ', regex=True, inplace=True)
#elimino los saltos de línea en la letra y temática de la canción

In [43]:
input_data

,id,name,artists,chords,instruments,speechiness,instrumentalness,key,mode,tempo,loudness,theme,chords_1,chords_2,chords_1_numeric,chords_2_numeric,normal_chords_1_numeric,normal_chords_2_numeric,diff_chords_1_numeric,diff_chords_2_numeric
0,42VgiRyg0YjtRVLiPFNxPJ,Zapatillas,El Canto Del Loco,"[C, G, Am, F, D, A, Bm, G]","[C, G, Am, F, F, G, C, Am]",0.178,0,8,0,198.076,-2.382,Estoy cansado de salir de noche y ver siempre ...,"[C, G, Am, F]","[D, A, Bm, G]","[0, 7, 21, 5]","[2, 9, 23, 7]","[-8, -1, 13, -3]","[-6, 1, 15, -1]","[7, 14, -16]","[7, 14, -16]"


Related tracks:

In [44]:
df_vectorial['chords_1'] = df_vectorial['chords'].apply(lambda x: x[:4]) 
df_vectorial['chords_2'] = df_vectorial['chords'].apply(lambda x: x[4:])
df_vectorial.drop(columns=['chords'], inplace=True)

In [45]:
df_vectorial.head(3)

,id,name,artists,instruments,speechiness,instrumentalness,key,mode,tempo,loudness,theme,chords_1,chords_2
0,2VSGzQqy9nqJefcYGJNWst,Tanto la Queria,Andy & Lucas,"[C, G, Am, F, C, G, F, C]",0.0605,0.000000,10,1,144.001,-5.718,"The theme of ""Tanto la Quería"" by Andy & Lucas...","[A, E, F#m, D]","[A, E, D, D]"
1,05iTWBqTlBZAYBzHql2B3Y,Kamikaze,Amaral,"[G, D, Em, C, C, G, Am, D]",0.0398,0.000047,8,0,137.770,-6.605,Para auyentar la soledad\nPara espantar la dec...,"[Am, G, F, C]","[C, G, Am, F]"
2,3f2k0XTc2ZXG17jEMDu8pp,Quiero tener tu presencia,Seguridad Social,"[D, A, Bm, G, D, A, G, A]",0.0797,0.000000,4,1,146.075,-5.597,"Quiero tener tu presencia, quiero que estés a ...","[C, G, Am, F]","[Dm, G, C, F]"


In [46]:
# Nos ocupamos de bemoles o sostenidos para estandarizarlos
df_vectorial['chords_1'] = df_vectorial['chords_1'].apply(standardize_chord_symbols)
df_vectorial['chords_2'] = df_vectorial['chords_2'].apply(standardize_chord_symbols)

In [47]:
df_vectorial.head(3)

,id,name,artists,instruments,speechiness,instrumentalness,key,mode,tempo,loudness,theme,chords_1,chords_2
0,2VSGzQqy9nqJefcYGJNWst,Tanto la Queria,Andy & Lucas,"[C, G, Am, F, C, G, F, C]",0.0605,0.000000,10,1,144.001,-5.718,"The theme of ""Tanto la Quería"" by Andy & Lucas...","[A, E, F#m, D]","[A, E, D, D]"
1,05iTWBqTlBZAYBzHql2B3Y,Kamikaze,Amaral,"[G, D, Em, C, C, G, Am, D]",0.0398,0.000047,8,0,137.770,-6.605,Para auyentar la soledad\nPara espantar la dec...,"[Am, G, F, C]","[C, G, Am, F]"
2,3f2k0XTc2ZXG17jEMDu8pp,Quiero tener tu presencia,Seguridad Social,"[D, A, Bm, G, D, A, G, A]",0.0797,0.000000,4,1,146.075,-5.597,"Quiero tener tu presencia, quiero que estés a ...","[C, G, Am, F]","[Dm, G, C, F]"


In [48]:
# Convertimos los acordes a valores numéricos
df_vectorial['chords_1_numeric'] = df_vectorial['chords_1'].apply(convert_chords_with_minor)
df_vectorial['chords_2_numeric'] = df_vectorial['chords_2'].apply(convert_chords_with_minor)

In [49]:
df_vectorial.head(3)

,id,name,artists,instruments,speechiness,instrumentalness,key,mode,tempo,loudness,theme,chords_1,chords_2,chords_1_numeric,chords_2_numeric
0,2VSGzQqy9nqJefcYGJNWst,Tanto la Queria,Andy & Lucas,"[C, G, Am, F, C, G, F, C]",0.0605,0.000000,10,1,144.001,-5.718,"The theme of ""Tanto la Quería"" by Andy & Lucas...","[A, E, F#m, D]","[A, E, D, D]","[9, 4, 18, 2]","[9, 4, 2, 2]"
1,05iTWBqTlBZAYBzHql2B3Y,Kamikaze,Amaral,"[G, D, Em, C, C, G, Am, D]",0.0398,0.000047,8,0,137.770,-6.605,Para auyentar la soledad\nPara espantar la dec...,"[Am, G, F, C]","[C, G, Am, F]","[21, 7, 5, 0]","[0, 7, 21, 5]"
2,3f2k0XTc2ZXG17jEMDu8pp,Quiero tener tu presencia,Seguridad Social,"[D, A, Bm, G, D, A, G, A]",0.0797,0.000000,4,1,146.075,-5.597,"Quiero tener tu presencia, quiero que estés a ...","[C, G, Am, F]","[Dm, G, C, F]","[0, 7, 21, 5]","[14, 7, 0, 5]"


In [50]:
col_chords = ['chords_1_numeric', 'chords_2_numeric']

# Aplicar la función a cada fila
for col in col_chords:
    df_vectorial[f'normal_{col}'] = df_vectorial.apply(lambda row: transpose_chords_to_key(row, col), axis=1)

df_vectorial

,id,name,artists,instruments,speechiness,instrumentalness,key,mode,tempo,loudness,theme,chords_1,chords_2,chords_1_numeric,chords_2_numeric,normal_chords_1_numeric,normal_chords_2_numeric
0,2VSGzQqy9nqJefcYGJNWst,Tanto la Queria,Andy & Lucas,"[C, G, Am, F, C, G, F, C]",0.0605,0.000000,10,1,144.001,-5.718,"The theme of ""Tanto la Quería"" by Andy & Lucas...","[A, E, F#m, D]","[A, E, D, D]","[9, 4, 18, 2]","[9, 4, 2, 2]","[-1, -6, 8, -8]","[-1, -6, -8, -8]"
1,05iTWBqTlBZAYBzHql2B3Y,Kamikaze,Amaral,"[G, D, Em, C, C, G, Am, D]",0.0398,0.000047,8,0,137.770,-6.605,Para auyentar la soledad\nPara espantar la dec...,"[Am, G, F, C]","[C, G, Am, F]","[21, 7, 5, 0]","[0, 7, 21, 5]","[13, -1, -3, -8]","[-8, -1, 13, -3]"
2,3f2k0XTc2ZXG17jEMDu8pp,Quiero tener tu presencia,Seguridad Social,"[D, A, Bm, G, D, A, G, A]",0.0797,0.000000,4,1,146.075,-5.597,"Quiero tener tu presencia, quiero que estés a ...","[C, G, Am, F]","[Dm, G, C, F]","[0, 7, 21, 5]","[14, 7, 0, 5]","[-4, 3, 17, 1]","[10, 3, -4, 1]"
3,4hTXeWayUVMwoz6v1CviSN,Copacabana,IZAL,"[C, G, Am, F, F, G, C, Am]",0.0485,0.000005,9,0,144.924,-7.194,"Antes de nada, dejaremos claras\nLas páginas q...","[F, A, D, B]","[G, C, Em, D]","[5, 9, 2, 11]","[7, 0, 16, 2]","[-4, 0, -7, 2]","[-2, -9, 7, -7]"


In [51]:
for col in col_chords:
    df_vectorial[f'diff_{col}'] = df_vectorial.apply(lambda row: calc_diff(row[col]), axis=1)

df_vectorial

,id,name,artists,instruments,speechiness,instrumentalness,key,mode,tempo,loudness,theme,chords_1,chords_2,chords_1_numeric,chords_2_numeric,normal_chords_1_numeric,normal_chords_2_numeric,diff_chords_1_numeric,diff_chords_2_numeric
0,2VSGzQqy9nqJefcYGJNWst,Tanto la Queria,Andy & Lucas,"[C, G, Am, F, C, G, F, C]",0.0605,0.000000,10,1,144.001,-5.718,"The theme of ""Tanto la Quería"" by Andy & Lucas...","[A, E, F#m, D]","[A, E, D, D]","[9, 4, 18, 2]","[9, 4, 2, 2]","[-1, -6, 8, -8]","[-1, -6, -8, -8]","[-5, 14, -16]","[-5, -2, 0]"
1,05iTWBqTlBZAYBzHql2B3Y,Kamikaze,Amaral,"[G, D, Em, C, C, G, Am, D]",0.0398,0.000047,8,0,137.770,-6.605,Para auyentar la soledad\nPara espantar la dec...,"[Am, G, F, C]","[C, G, Am, F]","[21, 7, 5, 0]","[0, 7, 21, 5]","[13, -1, -3, -8]","[-8, -1, 13, -3]","[-14, -2, -5]","[7, 14, -16]"
2,3f2k0XTc2ZXG17jEMDu8pp,Quiero tener tu presencia,Seguridad Social,"[D, A, Bm, G, D, A, G, A]",0.0797,0.000000,4,1,146.075,-5.597,"Quiero tener tu presencia, quiero que estés a ...","[C, G, Am, F]","[Dm, G, C, F]","[0, 7, 21, 5]","[14, 7, 0, 5]","[-4, 3, 17, 1]","[10, 3, -4, 1]","[7, 14, -16]","[-7, -7, 5]"
3,4hTXeWayUVMwoz6v1CviSN,Copacabana,IZAL,"[C, G, Am, F, F, G, C, Am]",0.0485,0.000005,9,0,144.924,-7.194,"Antes de nada, dejaremos claras\nLas páginas q...","[F, A, D, B]","[G, C, Em, D]","[5, 9, 2, 11]","[7, 0, 16, 2]","[-4, 0, -7, 2]","[-2, -9, 7, -7]","[4, -7, 9]","[-7, 16, -14]"


In [52]:
df_vectorial.replace('\n', ' ', regex=True, inplace=True)
#elimino los saltos de línea en la letra y temática de la canción

In [53]:
df_vectorial = df_vectorial.fillna(0)

In [54]:
df_vectorial.head(3)

,id,name,artists,instruments,speechiness,instrumentalness,key,mode,tempo,loudness,theme,chords_1,chords_2,chords_1_numeric,chords_2_numeric,normal_chords_1_numeric,normal_chords_2_numeric,diff_chords_1_numeric,diff_chords_2_numeric
0,2VSGzQqy9nqJefcYGJNWst,Tanto la Queria,Andy & Lucas,"[C, G, Am, F, C, G, F, C]",0.0605,0.000000,10,1,144.001,-5.718,"The theme of ""Tanto la Quería"" by Andy & Lucas...","[A, E, F#m, D]","[A, E, D, D]","[9, 4, 18, 2]","[9, 4, 2, 2]","[-1, -6, 8, -8]","[-1, -6, -8, -8]","[-5, 14, -16]","[-5, -2, 0]"
1,05iTWBqTlBZAYBzHql2B3Y,Kamikaze,Amaral,"[G, D, Em, C, C, G, Am, D]",0.0398,0.000047,8,0,137.770,-6.605,Para auyentar la soledad Para espantar la dece...,"[Am, G, F, C]","[C, G, Am, F]","[21, 7, 5, 0]","[0, 7, 21, 5]","[13, -1, -3, -8]","[-8, -1, 13, -3]","[-14, -2, -5]","[7, 14, -16]"
2,3f2k0XTc2ZXG17jEMDu8pp,Quiero tener tu presencia,Seguridad Social,"[D, A, Bm, G, D, A, G, A]",0.0797,0.000000,4,1,146.075,-5.597,"Quiero tener tu presencia, quiero que estés a ...","[C, G, Am, F]","[Dm, G, C, F]","[0, 7, 21, 5]","[14, 7, 0, 5]","[-4, 3, 17, 1]","[10, 3, -4, 1]","[7, 14, -16]","[-7, -7, 5]"


In [55]:
input_data

,id,name,artists,chords,instruments,speechiness,instrumentalness,key,mode,tempo,loudness,theme,chords_1,chords_2,chords_1_numeric,chords_2_numeric,normal_chords_1_numeric,normal_chords_2_numeric,diff_chords_1_numeric,diff_chords_2_numeric
0,42VgiRyg0YjtRVLiPFNxPJ,Zapatillas,El Canto Del Loco,"[C, G, Am, F, D, A, Bm, G]","[C, G, Am, F, F, G, C, Am]",0.178,0,8,0,198.076,-2.382,Estoy cansado de salir de noche y ver siempre ...,"[C, G, Am, F]","[D, A, Bm, G]","[0, 7, 21, 5]","[2, 9, 23, 7]","[-8, -1, 13, -3]","[-6, 1, 15, -1]","[7, 14, -16]","[7, 14, -16]"


# Backup csv

In [56]:
csv_file = 'mi_archivo_temporal.csv'

# Llamar la función para exportar
export_unique_rows(df_vectorial, csv_file)



/Users/gabrielacaseromonge/Documents/GitHub/song-sarter-streamlit/utils/functions.py:277: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_combined = df_combined.applymap(lambda x: str(x) if isinstance(x, list) else x)
/Users/gabrielacaseromonge/Documents/GitHub/song-sarter-streamlit/utils/functions.py:283: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_combined = df_combined.applymap(lambda x: eval(x) if isinstance(x, str) and x.startswith('[') and x.endswith(']') else x)


# Expand columns

In [57]:
# Especificar las columnas que contienen listas (excepto 'embedding')
columns_to_expand = ['instruments', 'chords_1', 'chords_2', 
                      'chords_1_numeric', 'chords_2_numeric', 
                      'normal_chords_1_numeric', 'normal_chords_2_numeric', 
                      'diff_chords_1_numeric', 'diff_chords_2_numeric']

Input:

In [58]:
input_data

,id,name,artists,chords,instruments,speechiness,instrumentalness,key,mode,tempo,loudness,theme,chords_1,chords_2,chords_1_numeric,chords_2_numeric,normal_chords_1_numeric,normal_chords_2_numeric,diff_chords_1_numeric,diff_chords_2_numeric
0,42VgiRyg0YjtRVLiPFNxPJ,Zapatillas,El Canto Del Loco,"[C, G, Am, F, D, A, Bm, G]","[C, G, Am, F, F, G, C, Am]",0.178,0,8,0,198.076,-2.382,Estoy cansado de salir de noche y ver siempre ...,"[C, G, Am, F]","[D, A, Bm, G]","[0, 7, 21, 5]","[2, 9, 23, 7]","[-8, -1, 13, -3]","[-6, 1, 15, -1]","[7, 14, -16]","[7, 14, -16]"


In [59]:
columns_to_expand

['instruments',
 'chords_1',
 'chords_2',
 'chords_1_numeric',
 'chords_2_numeric',
 'normal_chords_1_numeric',
 'normal_chords_2_numeric',
 'diff_chords_1_numeric',
 'diff_chords_2_numeric']

In [60]:
input_expanded = expand_and_remove_original_columns(input_data, columns_to_expand)

In [61]:
input_expanded

,id,name,artists,chords,speechiness,instrumentalness,key,mode,tempo,loudness,...,normal_chords_2_numeric_1,normal_chords_2_numeric_2,normal_chords_2_numeric_3,normal_chords_2_numeric_4,diff_chords_1_numeric_1,diff_chords_1_numeric_2,diff_chords_1_numeric_3,diff_chords_2_numeric_1,diff_chords_2_numeric_2,diff_chords_2_numeric_3
0,42VgiRyg0YjtRVLiPFNxPJ,Zapatillas,El Canto Del Loco,"[C, G, Am, F, D, A, Bm, G]",0.178,0,8,0,198.076,-2.382,...,-6,1,15,-1,7,14,-16,7,14,-16


In [62]:
#elimino las columnas que tengan solo valores nulos
input_expanded = input_expanded.dropna(axis=1, how='all')

Related tracks:

In [63]:
df_vectorial_expanded = expand_and_remove_original_columns(df_vectorial, columns_to_expand)

In [64]:
df_vectorial_expanded

,id,name,artists,speechiness,instrumentalness,key,mode,tempo,loudness,theme,...,normal_chords_2_numeric_1,normal_chords_2_numeric_2,normal_chords_2_numeric_3,normal_chords_2_numeric_4,diff_chords_1_numeric_1,diff_chords_1_numeric_2,diff_chords_1_numeric_3,diff_chords_2_numeric_1,diff_chords_2_numeric_2,diff_chords_2_numeric_3
0,2VSGzQqy9nqJefcYGJNWst,Tanto la Queria,Andy & Lucas,0.0605,0.000000,10,1,144.001,-5.718,"The theme of ""Tanto la Quería"" by Andy & Lucas...",...,-1,-6,-8,-8,-5,14,-16,-5,-2,0
1,05iTWBqTlBZAYBzHql2B3Y,Kamikaze,Amaral,0.0398,0.000047,8,0,137.770,-6.605,Para auyentar la soledad Para espantar la dece...,...,-8,-1,13,-3,-14,-2,-5,7,14,-16
2,3f2k0XTc2ZXG17jEMDu8pp,Quiero tener tu presencia,Seguridad Social,0.0797,0.000000,4,1,146.075,-5.597,"Quiero tener tu presencia, quiero que estés a ...",...,10,3,-4,1,7,14,-16,-7,-7,5
3,4hTXeWayUVMwoz6v1CviSN,Copacabana,IZAL,0.0485,0.000005,9,0,144.924,-7.194,"Antes de nada, dejaremos claras Las páginas qu...",...,-2,-9,7,-7,4,-7,9,-7,16,-14


In [65]:
#elimino las columnas que tengan solo valores nulos
df_vectorial_expanded = df_vectorial_expanded.dropna(axis=1, how='all')

In [66]:
drop_columns_with_many_nulls(df_vectorial_expanded, 15)

,id,name,artists,speechiness,instrumentalness,key,mode,tempo,loudness,theme,...,normal_chords_2_numeric_1,normal_chords_2_numeric_2,normal_chords_2_numeric_3,normal_chords_2_numeric_4,diff_chords_1_numeric_1,diff_chords_1_numeric_2,diff_chords_1_numeric_3,diff_chords_2_numeric_1,diff_chords_2_numeric_2,diff_chords_2_numeric_3
0,2VSGzQqy9nqJefcYGJNWst,Tanto la Queria,Andy & Lucas,0.0605,0.000000,10,1,144.001,-5.718,"The theme of ""Tanto la Quería"" by Andy & Lucas...",...,-1,-6,-8,-8,-5,14,-16,-5,-2,0
1,05iTWBqTlBZAYBzHql2B3Y,Kamikaze,Amaral,0.0398,0.000047,8,0,137.770,-6.605,Para auyentar la soledad Para espantar la dece...,...,-8,-1,13,-3,-14,-2,-5,7,14,-16
2,3f2k0XTc2ZXG17jEMDu8pp,Quiero tener tu presencia,Seguridad Social,0.0797,0.000000,4,1,146.075,-5.597,"Quiero tener tu presencia, quiero que estés a ...",...,10,3,-4,1,7,14,-16,-7,-7,5
3,4hTXeWayUVMwoz6v1CviSN,Copacabana,IZAL,0.0485,0.000005,9,0,144.924,-7.194,"Antes de nada, dejaremos claras Las páginas qu...",...,-2,-9,7,-7,4,-7,9,-7,16,-14


In [67]:
df_vectorial_expanded.head(3)

,id,name,artists,speechiness,instrumentalness,key,mode,tempo,loudness,theme,...,normal_chords_2_numeric_1,normal_chords_2_numeric_2,normal_chords_2_numeric_3,normal_chords_2_numeric_4,diff_chords_1_numeric_1,diff_chords_1_numeric_2,diff_chords_1_numeric_3,diff_chords_2_numeric_1,diff_chords_2_numeric_2,diff_chords_2_numeric_3
0,2VSGzQqy9nqJefcYGJNWst,Tanto la Queria,Andy & Lucas,0.0605,0.000000,10,1,144.001,-5.718,"The theme of ""Tanto la Quería"" by Andy & Lucas...",...,-1,-6,-8,-8,-5,14,-16,-5,-2,0
1,05iTWBqTlBZAYBzHql2B3Y,Kamikaze,Amaral,0.0398,0.000047,8,0,137.770,-6.605,Para auyentar la soledad Para espantar la dece...,...,-8,-1,13,-3,-14,-2,-5,7,14,-16
2,3f2k0XTc2ZXG17jEMDu8pp,Quiero tener tu presencia,Seguridad Social,0.0797,0.000000,4,1,146.075,-5.597,"Quiero tener tu presencia, quiero que estés a ...",...,10,3,-4,1,7,14,-16,-7,-7,5


# Create column for the text to create the embeddings

Input:

In [68]:
input_expanded=to_embbed_text_column(input_expanded)

In [69]:
input_expanded['to_embbed_text']

0    name: Zapatillas artists: El Canto Del Loco ch...
Name: to_embbed_text, dtype: object

Related songs:

In [70]:
df_vectorial_expanded=to_embbed_text_column(df_vectorial_expanded)

In [71]:
df_vectorial_expanded


,id,name,artists,speechiness,instrumentalness,key,mode,tempo,loudness,theme,...,normal_chords_2_numeric_2,normal_chords_2_numeric_3,normal_chords_2_numeric_4,diff_chords_1_numeric_1,diff_chords_1_numeric_2,diff_chords_1_numeric_3,diff_chords_2_numeric_1,diff_chords_2_numeric_2,diff_chords_2_numeric_3,to_embbed_text
0,2VSGzQqy9nqJefcYGJNWst,Tanto la Queria,Andy & Lucas,0.0605,0.000000,10,1,144.001,-5.718,"The theme of ""Tanto la Quería"" by Andy & Lucas...",...,-6,-8,-8,-5,14,-16,-5,-2,0,name: Tanto la Queria artists: Andy & Lucas sp...
1,05iTWBqTlBZAYBzHql2B3Y,Kamikaze,Amaral,0.0398,0.000047,8,0,137.770,-6.605,Para auyentar la soledad Para espantar la dece...,...,-1,13,-3,-14,-2,-5,7,14,-16,name: Kamikaze artists: Amaral speechiness: 0....
2,3f2k0XTc2ZXG17jEMDu8pp,Quiero tener tu presencia,Seguridad Social,0.0797,0.000000,4,1,146.075,-5.597,"Quiero tener tu presencia, quiero que estés a ...",...,3,-4,1,7,14,-16,-7,-7,5,name: Quiero tener tu presencia artists: Segur...
3,4hTXeWayUVMwoz6v1CviSN,Copacabana,IZAL,0.0485,0.000005,9,0,144.924,-7.194,"Antes de nada, dejaremos claras Las páginas qu...",...,-9,7,-7,4,-7,9,-7,16,-14,name: Copacabana artists: IZAL speechiness: 0....


# Text for embeddings to chat gpt

In [72]:
api_key = os.getenv("OPENAI_API_KEY")

# Configurar la clave de API en OpenAI
OpenAI.api_key = api_key

In [73]:
# Convert text into vectors using embeddings
client = OpenAI(
 api_key=api_key,
)

Input:

In [74]:
input_expanded['embedding'] = input_expanded['to_embbed_text'].apply(lambda x: client.embeddings.create(
model="text-embedding-3-small",
input=x
).data[0].embedding)

In [75]:
input_expanded.drop(columns=['to_embbed_text'], inplace=True)

In [76]:
df_vectorial_expanded

,id,name,artists,speechiness,instrumentalness,key,mode,tempo,loudness,theme,...,normal_chords_2_numeric_2,normal_chords_2_numeric_3,normal_chords_2_numeric_4,diff_chords_1_numeric_1,diff_chords_1_numeric_2,diff_chords_1_numeric_3,diff_chords_2_numeric_1,diff_chords_2_numeric_2,diff_chords_2_numeric_3,to_embbed_text
0,2VSGzQqy9nqJefcYGJNWst,Tanto la Queria,Andy & Lucas,0.0605,0.000000,10,1,144.001,-5.718,"The theme of ""Tanto la Quería"" by Andy & Lucas...",...,-6,-8,-8,-5,14,-16,-5,-2,0,name: Tanto la Queria artists: Andy & Lucas sp...
1,05iTWBqTlBZAYBzHql2B3Y,Kamikaze,Amaral,0.0398,0.000047,8,0,137.770,-6.605,Para auyentar la soledad Para espantar la dece...,...,-1,13,-3,-14,-2,-5,7,14,-16,name: Kamikaze artists: Amaral speechiness: 0....
2,3f2k0XTc2ZXG17jEMDu8pp,Quiero tener tu presencia,Seguridad Social,0.0797,0.000000,4,1,146.075,-5.597,"Quiero tener tu presencia, quiero que estés a ...",...,3,-4,1,7,14,-16,-7,-7,5,name: Quiero tener tu presencia artists: Segur...
3,4hTXeWayUVMwoz6v1CviSN,Copacabana,IZAL,0.0485,0.000005,9,0,144.924,-7.194,"Antes de nada, dejaremos claras Las páginas qu...",...,-9,7,-7,4,-7,9,-7,16,-14,name: Copacabana artists: IZAL speechiness: 0....


Related songs:

In [77]:
#para cada fila del dataframe, sacar el vector de embedding de la columna "to_embbed_text"
df_vectorial_expanded['embedding'] = df_vectorial_expanded['to_embbed_text'].apply(lambda x: client.embeddings.create(
model="text-embedding-3-small",
input=x
).data[0].embedding)

In [78]:
df_vectorial_expanded.drop(columns=['to_embbed_text'], inplace=True)

In [79]:
df_vectorial_expanded = df_vectorial_expanded.rename(columns={'embedding': 'values'})

# Data to pinecone

In [80]:

# Procesar el DataFrame
df_vectorial_expanded = create_metadata_column(df_vectorial_expanded)
df_vectorial_expanded = clean_metadata_column(df_vectorial_expanded)
df_para_pinecone = create_dataframe_for_pinecone(df_vectorial_expanded)
insert_into_pinecone(df_para_pinecone, index)



sending upsert requests:   0%|          | 0/4 [00:00<?, ?it/s]

In [81]:
df_vectorial_expanded

,id,name,artists,speechiness,instrumentalness,key,mode,tempo,loudness,theme,...,normal_chords_2_numeric_3,normal_chords_2_numeric_4,diff_chords_1_numeric_1,diff_chords_1_numeric_2,diff_chords_1_numeric_3,diff_chords_2_numeric_1,diff_chords_2_numeric_2,diff_chords_2_numeric_3,values,metadata
0,2VSGzQqy9nqJefcYGJNWst,Tanto la Queria,Andy & Lucas,0.0605,0.000000,10,1,144.001,-5.718,"The theme of ""Tanto la Quería"" by Andy & Lucas...",...,-8,-8,-5,14,-16,-5,-2,0,"[0.009778662584722042, 0.01847955211997032, -0...","{'name': 'Tanto la Queria', 'artists': 'Andy &..."
1,05iTWBqTlBZAYBzHql2B3Y,Kamikaze,Amaral,0.0398,0.000047,8,0,137.770,-6.605,Para auyentar la soledad Para espantar la dece...,...,13,-3,-14,-2,-5,7,14,-16,"[0.0284456517547369, 0.04549656808376312, -0.0...","{'name': 'Kamikaze', 'artists': 'Amaral', 'spe..."
2,3f2k0XTc2ZXG17jEMDu8pp,Quiero tener tu presencia,Seguridad Social,0.0797,0.000000,4,1,146.075,-5.597,"Quiero tener tu presencia, quiero que estés a ...",...,-4,1,7,14,-16,-7,-7,5,"[0.02511654980480671, 0.0007369312224909663, -...","{'name': 'Quiero tener tu presencia', 'artists..."
3,4hTXeWayUVMwoz6v1CviSN,Copacabana,IZAL,0.0485,0.000005,9,0,144.924,-7.194,"Antes de nada, dejaremos claras Las páginas qu...",...,7,-7,4,-7,9,-7,16,-14,"[0.031195348128676414, 0.043161071836948395, -...","{'name': 'Copacabana', 'artists': 'IZAL', 'spe..."


In [82]:
df_para_pinecone

,id,values,metadata
0,2VSGzQqy9nqJefcYGJNWst,"[0.009778662584722042, 0.01847955211997032, -0...","{'name': 'Tanto la Queria', 'artists': 'Andy &..."
1,05iTWBqTlBZAYBzHql2B3Y,"[0.0284456517547369, 0.04549656808376312, -0.0...","{'name': 'Kamikaze', 'artists': 'Amaral', 'spe..."
2,3f2k0XTc2ZXG17jEMDu8pp,"[0.02511654980480671, 0.0007369312224909663, -...","{'name': 'Quiero tener tu presencia', 'artists..."
3,4hTXeWayUVMwoz6v1CviSN,"[0.031195348128676414, 0.043161071836948395, -...","{'name': 'Copacabana', 'artists': 'IZAL', 'spe..."


# Query search in pinecone

In [83]:
response = realizar_consulta(client, input_expanded, index, top_k=5)

Resultado de la consulta: {'matches': [{'id': '1rqGuE3jvcvacQk96lfnl6',
              'metadata': {'artists': 'Geese',
                           'chords_1_1': 'G',
                           'chords_1_2': 'D',
                           'chords_1_3': 'Em',
                           'chords_1_4': 'C',
                           'chords_1_numeric_1': 7.0,
                           'chords_1_numeric_2': 2.0,
                           'chords_1_numeric_3': 16.0,
                           'chords_1_numeric_4': 0.0,
                           'chords_2_1': 'Am',
                           'chords_2_2': 'F',
                           'chords_2_3': 'C',
                           'chords_2_4': 'G',
                           'chords_2_numeric_1': 21.0,
                           'chords_2_numeric_2': 5.0,
                           'chords_2_numeric_3': 0.0,
                           'chords_2_numeric_4': 7.0,
                           'diff_chords_1_numeric_1': -5.0,
                 

In [84]:
response_matches = response['matches']

details = []
for match in response_matches:
    details.append(f"Intruments: {match['metadata']['instruments_1']} {match['metadata']['instruments_2']}, Tempo: {match['metadata']['tempo']}, Speechiness: {match['metadata']['speechiness']}, Loudness: {match['metadata']['loudness']}")

In [85]:
details

['Intruments: guitar bass, Tempo: 131.948, Speechiness: 0.0382, Loudness: -7.699',
 'Intruments: piano cello, Tempo: 133.355, Speechiness: 0.033, Loudness: -12.636',
 'Intruments: guitar saxophone, Tempo: 131.685, Speechiness: 0.0469, Loudness: -4.09',
 'Intruments: piano guitar, Tempo: 114.032, Speechiness: 0.0388, Loudness: -6.153',
 'Intruments: guitar drums, Tempo: 117.252, Speechiness: 0.07, Loudness: -7.16']

In [86]:

for match in response_matches:
    print(f"Letra: {match['metadata']['theme']}, \n Score: {match['score']}")

Letra: I’ve been hit by the bus of love That falling brick of you Baby, when there’s tears in your eyes You don’t have to lie, 
 Score: 0.204969212
Letra: Love and learn Take your turn Somewhere there's a feather Falling slowly from the sky, 
 Score: 0.197386891
Letra: The song "El Encuentro" by Biela explores themes of connection, longing, and intimacy. It captures the emotions surrounding a significant meeting between two individuals, emphasizing the beauty and complexity of human relationships. Through its lyrical imagery, it conveys a sense of urgency and the profound impact such encounters can have on one’s life. Overall, the song reflects on the joys and challenges of love and the deep bonds formed through shared experiences., 
 Score: 0.194683358
Letra: The theme of "Miles and Miles of Texas" revolves around the deep connection to the vast Texas landscape and the sense of freedom it brings to the narrator. It reflects a longing for home and the cherished memories tied to the pla

In [87]:
diff_chord_2= input_data['diff_chords_2_numeric'].iloc[0]
diff_chord_1= input_data['diff_chords_1_numeric'].iloc[0]

In [88]:
# Crear el embedding de la nueva query
chords_query = f"A song with a similar progession and distance (diff) in chords to {input_data['chords_1'][0]},y diff chords {diff_chord_2},{diff_chord_1}."
theme_embedding = client.embeddings.create(
    model="text-embedding-3-small",
    input=chords_query
).data[0].embedding

chords_query = index.query(vector=theme_embedding, top_k=5, include_metadata=True)

chords_query_ = chords_query['matches']



In [89]:
for match in chords_query_:
    print(f"Chords: {match['metadata']['chords_1_1']} {match['metadata']['chords_1_2']} {match['metadata']['chords_1_3']} {match['metadata']['chords_1_4']}, {match['metadata']['chords_2_1']} {match['metadata']['chords_2_2']} {match['metadata']['chords_2_3']} {match['metadata']['chords_2_4']}, \n Score: {match['score']}")

Chords: C G Am F, F Am C G, 
 Score: 0.465003759
Chords: C G Am F, C F G Am, 
 Score: 0.45702365
Chords: C G Am F, C G F F, 
 Score: 0.452040702
Chords: 2023 Remaster C G Am, F C G F, 
 Score: 0.451456219
Chords: C G Am F, D A Bm G, 
 Score: 0.448484957


In [90]:
chord_wheels=[]
for match in chords_query_:
    chord_wheels.append(f"Chord wheel: {match['metadata']['chords_1_1']} {match['metadata']['chords_1_2']} {match['metadata']['chords_1_3']} {match['metadata']['chords_1_4']}, {match['metadata']['chords_2_1']} {match['metadata']['chords_2_2']} {match['metadata']['chords_2_3']} {match['metadata']['chords_2_4']}")

In [91]:
chord_wheels

['Chord wheel: C G Am F, F Am C G',
 'Chord wheel: C G Am F, C F G Am',
 'Chord wheel: C G Am F, C G F F',
 'Chord wheel: 2023 Remaster C G Am, F C G F',
 'Chord wheel: C G Am F, D A Bm G']

In [92]:
input_data['theme'][0]

'Estoy cansado de salir de noche y ver siempre la misma gente. Estoy flipando de que la gente se invente, cuente y luego reinvente. Apotronado en el sofá de mi casa, vente se está caliente. Amaestrados vamos al mismo sitio todos aunque luego ni entres. Alucinando de que me miren de arriba a bajo como un delincuente. Intoxicado de que me pongan esa puta música indiferente.  (ESTRIBILLO)'

In [93]:
# Crear el embedding de la nueva query
lyrics_query = f"Give me a similar theme song to {input_data['theme'][0]}"
theme_embedding = client.embeddings.create(
    model="text-embedding-3-small",
    input=lyrics_query
).data[0].embedding

lyrics_result = index.query(vector=theme_embedding, top_k=5, include_metadata=True)

lyrics_matches = lyrics_result['matches']



In [94]:
lyrics_list = []  # Crear una lista vacía para almacenar los textos

for match in lyrics_matches:
    # Agregar cada texto generado a la lista
    lyrics_list.append(f"Lyrics: {match['metadata']['theme']}")



for match in lyrics_matches:
    # Agregar cada texto generado a la lista
    print(f"Lyrics: {match['metadata']['theme']} {match['id']} Score: {match['score']}")

lyrics_list.append(input_data['theme'][0])

Lyrics: Estoy cansado de salir de noche y ver siempre la misma gente. Estoy flipando de que la gente se invente, cuente y luego reinvente. Apotronado en el sofá de mi casa, vente se está caliente. Amaestrados vamos al mismo sitio todos aunque luego ni entres. Alucinando de que me miren de arriba a bajo como un delincuente. Intoxicado de que me pongan esa puta música indiferente.  (ESTRIBILLO) 42VgiRyg0YjtRVLiPFNxPJ Score: 0.65762794
Lyrics: Dime, que la noche aqui no acaba que me quedo como estaba con las ganas de volar. Dime, que me subo por las ramas la sangre se me derrama  y estoy arto de aguantar.  7p86AB1S0CeNeYc7CfWvGR Score: 0.539999
Lyrics: Dónde estabas entonces cuando tanto te necesité? Nadie es mejor que nadie pero tu creíste vencer. Si lloré ante tu puerta de nada sirvió. Barras de bar, vertederos de amor... Os enseñé mi trocito peor.  Retales de mi vida,  6xnHAQ96sirlw7Gm4ppqxG Score: 0.527094543
Lyrics: No te quiero ver más, te he dicho que me dejes en paz que no quedo c

In [95]:
lyrics_list

['Lyrics: Estoy cansado de salir de noche y ver siempre la misma gente. Estoy flipando de que la gente se invente, cuente y luego reinvente. Apotronado en el sofá de mi casa, vente se está caliente. Amaestrados vamos al mismo sitio todos aunque luego ni entres. Alucinando de que me miren de arriba a bajo como un delincuente. Intoxicado de que me pongan esa puta música indiferente.  (ESTRIBILLO)',
 'Lyrics: Dime, que la noche aqui no acaba que me quedo como estaba con las ganas de volar. Dime, que me subo por las ramas la sangre se me derrama  y estoy arto de aguantar. ',
 'Lyrics: Dónde estabas entonces cuando tanto te necesité? Nadie es mejor que nadie pero tu creíste vencer. Si lloré ante tu puerta de nada sirvió. Barras de bar, vertederos de amor... Os enseñé mi trocito peor.  Retales de mi vida, ',
 'Lyrics: No te quiero ver más, te he dicho que me dejes en paz que no quedo contigo esta tarde que te largues ya que no quiero más que te vayas al cine que vuelvas jamás a llamarme  ',


In [96]:
nueva_cancion = create_song(lyrics_list, chord_wheels, details)
print(nueva_cancion)

# Título de la canción: 
Noches Repetidas

**Tempo:** 131 BPM  
**Instrumentos principales:** guitarra, batería

### Letra y acordes:

1. **Verso 1**:
   - [C] Camino solo por la [G] calle vacía,  
   - [Am] buscando algo que [F] nunca termina.  
   - [C] Palmas conocidas se [G] pierden en la multitud,  
   - [Am] mientras busco un [F] nuevo latitud.  

2. **Coro**:
   - [C] Y estoy harto, de [G] repetir cada paso,  
   - [Am] del ruido que [F] suena en falso.  
   - [C] Noches que se [G] rompen en vasos,  
   - [Am] y yo ya no [F] quiero más descanso.  

3. **Verso 2**:
   - [C] Miradas que juzgan [G] cada movimiento,  
   - [Am] Música que ya [F] no siento.  
   - [C] Historias que [G] nadie ha vivido,  
   - [Am] en la rutina de [F] lo conocido.  

4. **Coro final**:
   - [C] Y estoy harto, de [G] no encontrar un espacio,  
   - [Am] donde el ruido [F] deje de ser escaso.  
   - [C] Noches que se [G] parten en pasos,  
   - [Am] y yo voy a [F] romper el lazo.  

**Nota**: La canción

In [97]:
nueva_cancion

'# Título de la canción: \nNoches Repetidas\n\n**Tempo:** 131 BPM  \n**Instrumentos principales:** guitarra, batería\n\n### Letra y acordes:\n\n1. **Verso 1**:\n   - [C] Camino solo por la [G] calle vacía,  \n   - [Am] buscando algo que [F] nunca termina.  \n   - [C] Palmas conocidas se [G] pierden en la multitud,  \n   - [Am] mientras busco un [F] nuevo latitud.  \n\n2. **Coro**:\n   - [C] Y estoy harto, de [G] repetir cada paso,  \n   - [Am] del ruido que [F] suena en falso.  \n   - [C] Noches que se [G] rompen en vasos,  \n   - [Am] y yo ya no [F] quiero más descanso.  \n\n3. **Verso 2**:\n   - [C] Miradas que juzgan [G] cada movimiento,  \n   - [Am] Música que ya [F] no siento.  \n   - [C] Historias que [G] nadie ha vivido,  \n   - [Am] en la rutina de [F] lo conocido.  \n\n4. **Coro final**:\n   - [C] Y estoy harto, de [G] no encontrar un espacio,  \n   - [Am] donde el ruido [F] deje de ser escaso.  \n   - [C] Noches que se [G] parten en pasos,  \n   - [Am] y yo voy a [F] romper e